In [1]:
!pip install ase

In [2]:
from google.colab import files
def getLocalFiles():
    _files = files.upload()
    if len(_files) >0:
       for k,v in _files.items():
         open(k,'wb').write(v)
getLocalFiles()
import structures
import simulator

data = simulator.simulate_all(simulator.presets['set_A'])

Saving inspect_data.ipynb to inspect_data (1).ipynb
Saving simulator.py to simulator (2).py
Saving structures.py to structures (2).py
Saving test_simulate_image.ipynb to test_simulate_image (1).ipynb


100%|██████████| 400/400 [00:04<00:00, 90.84it/s]


the next cell is just in case, the data could not be loaded by the cell above

In [17]:
import os
from pathlib import Path

import numpy as np
from ase.lattice.hexagonal import Graphene
from scipy.interpolate import interp1d
from scipy.ndimage import gaussian_filter
from structures import cut_rectangle
from tqdm import tqdm


def simulate_2d_material(atoms, shape, probe_profile, power_law):
    """
    Simulate a STEM image of a 2d material using the convolution approximation.

    Parameters
    ----------
    atoms : ASE Atoms object
        The 2d structure to simulate.
    shape : two ints
        The shape of the output image.
    probe_profile : Callable
        Function for calculating the probe profile.
    power_law : float
        The assumed Z-contrast powerlaw

    Returns
    -------
    ndarray
        Simulated STEM image.
    """

    extent = np.diag(atoms.cell)[:2]
    sampling = extent / shape

    margin = int(np.ceil(probe_profile.x[-1] / min(sampling)))
    shape_w_margin = (shape[0] + 2 * margin, shape[1] + 2 * margin)

    x = np.fft.fftfreq(shape_w_margin[0]) * shape_w_margin[1] * sampling[0]
    y = np.fft.fftfreq(shape_w_margin[1]) * shape_w_margin[1] * sampling[1]

    r = np.sqrt(x[:, None] ** 2 + y[None] ** 2)
    intensity = probe_profile(r)

    positions = atoms.positions[:, :2] / sampling

    inside = ((positions[:, 0] > -margin) &
              (positions[:, 1] > -margin) &
              (positions[:, 0] < shape[0] + margin) &
              (positions[:, 1] < shape[1] + margin))

    positions = positions[inside] + margin - .5

    array = np.zeros(shape_w_margin)
    for number in np.unique(atoms.numbers):
        temp = np.zeros(shape_w_margin)
        superpose_deltas(positions[atoms.numbers == number], temp)
        array += temp * number ** power_law

    array = np.fft.ifft2(np.fft.fft2(array) * np.fft.fft2(intensity)).real
    array = array[margin:-margin, margin:-margin]
    return array


def superpose_deltas(positions: np.ndarray, array: np.ndarray):
    """ Superpose delta functions """
    shape = array.shape[-2:]
    rounded = np.floor(positions).astype(np.int32)
    rows, cols = rounded[:, 0], rounded[:, 1]

    array[rows, cols] += (1 - (positions[:, 0] - rows)) * (1 - (positions[:, 1] - cols))
    array[(rows + 1) % shape[0], cols] += (positions[:, 0] - rows) * (1 - (positions[:, 1] - cols))
    array[rows, (cols + 1) % shape[1]] += (1 - (positions[:, 0] - rows)) * (positions[:, 1] - cols)
    array[(rows + 1) % shape[0], (cols + 1) % shape[1]] += (rows - positions[:, 0]) * (cols - positions[:, 1])


def make_random_hbn_model(extent):
    hbn = Graphene(symbol='N', latticeconstant={'a': 2.502, 'c': 12})
    hbn[0].symbol = 'B'
    rotation = np.random.rand() * 360
    hbn.rotate(rotation, 'z', rotate_cell=True)
    hbn = cut_rectangle(hbn, (0, 0), extent, margin=5)
    return hbn


def add_vacancy(atoms, number, atomic_number=None, margin=0):
    inside = ((atoms.positions[:, 0] > margin) &
              (atoms.positions[:, 1] > margin) &
              (atoms.positions[:, 0] < atoms.cell[0, 0] - margin) &
              (atoms.positions[:, 1] < atoms.cell[1, 1] - margin))

    if atomic_number is not None:
        inside = inside & (atoms.numbers == atomic_number)

    del atoms[np.random.choice(np.where(inside)[0], number, replace=False)]


def make_probe():
    gaussian = lambda x, sigma: np.exp(-x ** 2 / (2 * sigma ** 2))
    lorentz = lambda x, gamma: gamma / (np.pi * (x ** 2 + gamma ** 2))

    x = np.linspace(0, 5, 100)
    profile = gaussian(x, .4) + lorentz(x, 1)
    return interp1d(x, profile, fill_value=0, bounds_error=False)


def add_contamination(image, amount):
    if amount > 0:
        low_frequency_noise = gaussian_filter(np.random.randn(*image.shape), 10)
        low_frequency_noise -= low_frequency_noise.min()
        image += amount * low_frequency_noise


def add_noise(image, amount):
    if amount > 0:
        image[:] = np.random.poisson(image / amount).astype(np.float) * amount


presets = {'set_A':
               {'num_examples': 4000,  # Total number of examples to simulate, the test set will be 10 % of these
                'num_pixels': 48,  # Image size in pixels
                'fov': 15,  # Field of view in Angstrom
                'contamination': 0,  # Scale amount of mobile contaminants, realistic values in 0 to 100
                'noise': 0,  # Scale amount of noise, realistic values in 0 to 2
                'labels': 'basic',  # The labelling scheme, must be 'basic' or 'detailed'
                'margin': 1.5 * 2.502,  # No vacancies within this distance of the image edge (in Angstrom)
                },
           'set_B':
               {'num_examples': 4000,
                'num_pixels': 48,
                'fov': 15,
                'contamination': 0,
                'noise': 0,
                'labels': 'detailed',
                'margin': 1.5 * 2.502,
                }
           }


def simulate_all(preset):
    shape = (preset['num_pixels'],) * 2
    contamination = preset['contamination']
    noise = preset['noise']
    extent = (preset['fov'],) * 2

    for prefix in ('train', 'test'):

        if prefix == 'train':
            N = int(preset['num_examples'] * .9)
        else:
            N = preset['num_examples'] - int(preset['num_examples'] * .9)

        images = np.zeros((N,) + shape, dtype=np.float32)
        labels = np.zeros(N, dtype=np.int)

        for i in tqdm(range(N)):
            num_b_vacancies = np.random.poisson(.4)
            num_n_vacancies = np.random.poisson(.4)

            atoms = make_random_hbn_model(extent)

            add_vacancy(atoms, num_b_vacancies, 5, preset['margin'])
            add_vacancy(atoms, num_n_vacancies, 7, preset['margin'])

            probe = make_probe()
            image = simulate_2d_material(atoms, shape, probe, 1.6)

            add_contamination(image, contamination)
            add_noise(image, noise)

            if (num_b_vacancies + num_n_vacancies) == 0:
                label = 0
            elif (num_b_vacancies == 1) & (num_n_vacancies == 0) & (preset['labels'] == 'detailed'):
                label = 1
            elif (num_b_vacancies == 0) & (num_n_vacancies == 1) & (preset['labels'] == 'detailed'):
                label = 2
            else:
                if (preset['labels'] == 'detailed'):
                    label = 3
                else:
                    label = 1

            images[i] = ((image - image.mean()) / image.std()).astype(np.float32)
            labels[i] = label

    return images, labels


if __name__ == '__main__':

    # Choose a preset here
    preset_key = 'set_A'

    preset = presets[preset_key]

    shape = (preset['num_pixels'],) * 2
    contamination = preset['contamination']
    noise = preset['noise']
    extent = (preset['fov'],) * 2
    folder = os.path.join(os.path.abspath('..'), 'data', preset_key)

    Path(folder).mkdir(parents=True, exist_ok=True)

    for prefix in ('train', 'test'):

        if prefix == 'train':
            N = int(preset['num_examples'] * .9)
        else:
            N = preset['num_examples'] - int(preset['num_examples'] * .9)

        images = np.zeros((N,) + shape, dtype=np.float32)
        labels = np.zeros(N, dtype=np.int)

        for i in tqdm(range(N)):
            num_b_vacancies = np.random.poisson(.4)
            num_n_vacancies = np.random.poisson(.4)

            atoms = make_random_hbn_model(extent)

            add_vacancy(atoms, num_b_vacancies, 5, preset['margin'])
            add_vacancy(atoms, num_n_vacancies, 7, preset['margin'])

            probe = make_probe()
            image = simulate_2d_material(atoms, shape, probe, 1.6)

            add_contamination(image, contamination)
            add_noise(image, noise)

            if (num_b_vacancies + num_n_vacancies) == 0:
                label = 0
            elif (num_b_vacancies == 1) & (num_n_vacancies == 0) & (preset['labels'] == 'detailed'):
                label = 1
            elif (num_b_vacancies == 0) & (num_n_vacancies == 1) & (preset['labels'] == 'detailed'):
                label = 2
            else:
                if (preset['labels'] == 'detailed'):
                    label = 3
                else:
                    label = 1

            images[i] = ((image - image.mean()) / image.std()).astype(np.float32)
            labels[i] = label

        np.save(os.path.join(folder, '_'.join((prefix, 'images.npy'))), images)
        np.save(os.path.join(folder, '_'.join((prefix, 'labels.npy'))), labels)



100%|██████████| 400/400 [00:04<00:00, 90.47it/s]


In [3]:
#define the class

import torch
import torch.nn as nn 
import torch.nn.functional as F
import numpy as np

class Net(nn.Module):
    def __init__(self):
        super(Net,self).__init__()
        self.conv1=nn.Conv2d(1,10,3)
        self.pool=nn.MaxPool2d(2,2)
        self.conv2=nn.Conv2d(10,16,3)
        self.fc1=nn.Linear(16*10*10, 120)
        self.fc2=nn.Linear(120, 84)
        self.fc3=nn.Linear(84, 2)
        
    def forward(self,x):
        x=self.pool(F.relu(self.conv1(x)))
        x=self.pool(F.relu(self.conv2(x)))
        #print(x.size())
        x=x.view(-1,16*10*10)
        x=F.relu(self.fc1(x))
        x=F.relu(self.fc2(x))
        x=self.fc3(x)
        
        return x
        
net=Net()


In [4]:
# define optimizer and loss

import torch.optim as optim

loss_function=nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(), lr=0.001, momentum = 0.9)


In [5]:
# data from set_A
#import simulator
#import structures
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.ToTensor()])

all_images = np.load('../data/set_A/train_images.npy')[:, None]
all_labels = np.load('../data/set_A/train_labels.npy')

data = [(image, label) for image, label in zip(all_images, all_labels)]

trainloader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True, num_workers=0)

all_images = np.load('../data/set_A/test_images.npy')[:, None]
all_labels = np.load('../data/set_A/test_labels.npy')

data = [(image, label) for image, label in zip(all_images, all_labels)]
testloader = torch.utils.data.DataLoader(data, batch_size=4, shuffle=True, num_workers=0)

classes = ('pristine', 'defect')

In [6]:
# training the nn

EPOCHS=5
for epoch in range(EPOCHS):
    
    running_loss=0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels= data
        optimizer.zero_grad()
        outputs=net(inputs)
        #print(i)
        #print(outputs)
        #print(inputs)
        loss=loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss+= loss.item()
        if i%200==199:
            print("[%d,%5d] loss: %.3f" % (epoch+1, i+1, running_loss/2000))
            running_loss = 0.0
print("Finished Training")

[1,  200] loss: 0.069
[1,  400] loss: 0.069
[1,  600] loss: 0.069
[1,  800] loss: 0.068
[2,  200] loss: 0.057
[2,  400] loss: 0.026
[2,  600] loss: 0.010
[2,  800] loss: 0.004
[3,  200] loss: 0.001
[3,  400] loss: 0.001
[3,  600] loss: 0.000
[3,  800] loss: 0.000
[4,  200] loss: 0.000
[4,  400] loss: 0.000
[4,  600] loss: 0.000
[4,  800] loss: 0.000
[5,  200] loss: 0.000
[5,  400] loss: 0.000
[5,  600] loss: 0.000
[5,  800] loss: 0.000
Finished Training


In [7]:
# show the accuracy

correct=0
total=0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs= net(images)
        _, predicted=torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted==labels).sum().item()
print("Accuracy of the network: %d %%" %(100*correct/total))

Accuracy of the network: 100 %


In [8]:
# shows the accuracy of each class

class_correct=list(0. for i in range(2))
class_total=list(0. for i in range(2))
with torch.no_grad():
    for data in testloader:
        images, labels =data
        outputs= net(images)
        _, predicted=torch.max(outputs, 1)
        c= (predicted==labels).squeeze()
        for i in range(4):
            label=labels[i]
            class_correct[label]+= c[i].item()
            class_total[label]+=1

for i in range(2):
    print("Accuracy of %5s : %2d %%" %(classes[i], 100*class_correct[i]/class_total[i]))

Accuracy of pristine : 100 %
Accuracy of defect : 100 %
